In [ ]:
import pandas as pd

# Load CSV files
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

# Add labels
fake['label'] = 'Fake'
true['label'] = 'Real'

# Combine and shuffle
data = pd.concat([fake, true])
data = pd.concat([fake, true]).sample(frac=1, random_state=42).reset_index(drop=True)

# Keep only the columns we need
data = data[['text', 'label']]

# Preview first few rows
data.head()



,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",Fake
1,WASHINGTON (Reuters) - U.S. President Donald T...,Real
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,Real
3,"On Monday, Donald Trump once again embarrassed...",Fake
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",Real


In [ ]:
import re

# Define a text cleaning function
def clean_text(text):
    text = str(text).lower()                  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)           # Remove punctuation
    text = re.sub(r'\s+', ' ', text)          # Remove extra whitespace
    return text

# Apply the cleaning to all text entries
data['text'] = data['text'].apply(clean_text)

# Preview cleaned data
data.head()
data['label'].value_counts()



,count
label,
Fake,23481
Real,21417


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_df=0.7,
    min_df=3,               # Remove very rare words
    ngram_range=(1, 3),     # Include unigrams, bigrams, trigrams
    sublinear_tf=True       # Normalize term frequency
)

X = vectorizer.fit_transform(data['text'])
y = data['label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)




LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Show accuracy
print(f" Model Accuracy: {accuracy * 100:.2f}%")


 Model Accuracy: 98.92%


In [ ]:
model.classes_


array(['Fake', 'Real'], dtype=object)

In [ ]:
def predict_news_debug(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    probs = model.predict_proba(vec)[0]
    return {
        'Fake': f"{probs[model.classes_.tolist().index('Fake')]*100:.2f}%",
        'Real': f"{probs[model.classes_.tolist().index('Real')]*100:.2f}%"
    }


In [ ]:
predict_news_debug(data[data['label'] == 'Real'].iloc[10]['text'])


{'Fake': '12.53%', 'Real': '87.47%'}

In [ ]:
import gradio as gr
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Prediction function
def predict_news_bert(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    labels = ["Fake", "Real"]
    pred_index = torch.argmax(probs).item()
    confidence = probs[pred_index].item() * 100
    return f"📰 Prediction: {labels[pred_index]} ({confidence:.2f}% confidence)"

# Gradio UI
demo = gr.Interface(
    fn=predict_news_bert,
    inputs=gr.Textbox(lines=8, placeholder="Paste news article here...", label="News Article"),
    outputs=gr.Textbox(label="Prediction"),
    title="🧠 BERT Powered Fake News Detector",
    description="Paste any news article to check whether it's Fake or Real using a BERT model."
)

# Launch
demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52f200e745b3aa29fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# Step 4: Create project folder and go inside it
!mkdir FakeNewsApp
%cd FakeNewsApp

# Create app.py with your BERT + Gradio code
app_code = """
import gradio as gr
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

def predict_news_bert(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    labels = ["Fake", "Real"]
    pred_index = torch.argmax(probs).item()
    confidence = probs[pred_index].item() * 100
    return f"📰 Prediction: {labels[pred_index]} ({confidence:.2f}% confidence)"

demo = gr.Interface(
    fn=predict_news_bert,
    inputs=gr.Textbox(lines=8, label="News Article"),
    outputs=gr.Textbox(label="Prediction"),
    title=" BERT Fake News Detector",
    description="Paste a news article and find out if it's Fake or Real using BERT!"
)

demo.launch()
"""

with open("app.py", "w") as f:
    f.write(app_code)

# Create requirements.txt
with open("requirements.txt", "w") as f:
    f.write("gradio\ntransformers\ntorch\n")


/content/FakeNewsApp


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# ✅ This is your actual space ID
repo_id = "indhupamula/fake-news-bert-indhu"

# Upload app.py
api.upload_file(
    path_or_fileobj="/content/FakeNewsApp/app.py",
    path_in_repo="app.py",
    repo_id=repo_id,
    repo_type="space"
)

# Upload requirements.txt
api.upload_file(
    path_or_fileobj="/content/FakeNewsApp/requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=repo_id,
    repo_type="space"
)


CommitInfo(commit_url='https://huggingface.co/spaces/indhupamula/fake-news-bert-indhu/commit/600d310f3007e438ad25c18e11bbb90837230bde', commit_message='Upload requirements.txt with huggingface_hub', commit_description='', oid='600d310f3007e438ad25c18e11bbb90837230bde', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/indhupamula/fake-news-bert-indhu', endpoint='https://huggingface.co', repo_type='space', repo_id='indhupamula/fake-news-bert-indhu'), pr_revision=None, pr_num=None)